In [27]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import datetime
import dateutil
import pandas as pd
import numpy as np

In [28]:
def fetch_js_page(url,driver,sleep_time=5):
    driver.get(url)
    time.sleep(sleep_time)
    page_source=driver.page_source
    soup=BeautifulSoup(page_source,'lxml')    
    return soup

def fetch_overview_url(soup):
    #
    links=soup.find_all('a')
    
    for link in links:
    
        if 'overview' in str(link):
            url=link['href']    
    
    return url

#
def build_overview_soup():
    url="view-source:https://www.bmo.com/gam/ca/advisor/products/etfs#--tabs-1596907295545-"
    # fetch product information
    soup=fetch_js_page(url,driver,sleep_time=5)
    # fetch overview URL
    overview_url=fetch_overview_url(soup)
    # build overview soup
    overview_soup=fetch_js_page(str(overview_url),driver,sleep_time=5)

    return overview_soup

def fetch_overview_csv_url(overview_soup):
    for cell in overview_soup.select('#listOverview > div.table-top-wrapper > div > a:nth-child(2)'):
        overview_download_csv_url=cell['href']
    
    return overview_download_csv_url

def fetch_fund_url_list(overview_soup,base_url):

    links=overview_soup.find_all('a')

    fund_list=list()

    for link in links:
        if 'fundProfile/' in str(link):
            # extract fund URL
            fund_url=link['href'].strip()
            # extract fund name
            fund_name=link.text.strip()
            fund_list.append([fund_name,base_url+fund_url])
    
    return fund_list

def extract_fund_ticker(fund_url):
    
    fund_ticker=fund_url.replace("https://api.kurtosys.io/tools/ksys317/fundProfile/",
        "").replace("?locale=en-CA","")
    
    return fund_ticker

def fetch_json():
    d = json.loads(requests.get('https://api.kurtosys.io/entity/get?_api_key=EA87F964-5384-484A-954B-18C394A23126&_user_token=80AC692F-1E69-47FF-B0DC-91BC94DF29C2&entitytypeid=4617&getallforclient=true&maxresults=1000&admindata=0').text)['data']['value']
    
    return d

def fetch_fund_id(lookingFor,d):
    found = None
    for id, entry in d.items():
        for idInfo in entry['identifiers'].values():
            if idInfo['identifier'] == lookingFor:
                found = id
                break
        if found is not None:
            break
    
    return found

def fetch_fund_holdings_csv_url(fund_ticker,fund_id):

    holdings_csv_url='https://api.kurtosys.io/tools/ksys317/api/download/csv/holding/%s?fileName=%s_Holdings.csv&locale=en-CA' % (fund_id, fund_ticker)
    print(holdings_csv_url)
                
    return holdings_csv_url


def fetch_fund_return_csv_url(fund_ticker,fund_id):
        
    return_csv_url='https://api.kurtosys.io/tools/ksys317/api/download/csv/indexReturn/%s?fileName=%s_ETF_IndexReturns.csv&locale=en-CA' % (fund_id, fund_ticker)
    print(return_csv_url)
                
    return return_csv_url


def fetch_fund_nav_csv_url(fund_ticker,fund_id):
                
    nav_csv_url='https://api.kurtosys.io/tools/ksys317/api/download/csv/historicalNav/%s?fileName=%s_HistoricalNAV.csv&locale=en-CA' % (fund_id, fund_ticker)
    print(nav_csv_url)
        
    return nav_csv_url

def downloadFile(url,outputDirectory,outputFileName):

    # start a session
    s = requests.session()

    # set the parameters
    parameters = {'siteEntryPassthrough': True}
 
    # try to extract the data
    try:
        # fetch the data
        r = s.get(url, params=parameters)                              
        # open the output file handle                                       
        outputFileHandle = open(outputDirectory+outputFileName,'wb')
        # write the holdings
        outputFileHandle.write(r.content)
        # close the file handle
        outputFileHandle.close()
                
    except Exception: 
        pass    
        
    return

def build_fund_master_df(overview_soup,base_url):
    # fetch fund URL list
    fund_list=fetch_fund_url_list(overview_soup,base_url)
    # create fund list dataframe
    fund_name_2_url_df=pd.DataFrame(fund_list,columns=['fund_name','fund_url'])
    
    return fund_name_2_url_df

def download_fund_CSVs(fund_name_2_url_df,directory_dict):
    runDate=datetime.datetime.now()    
    # extract directories
    holdings_directory=directory_dict['holdings']
    nav_directory=directory_dict['nav']
    # iterate over each fund and download holdings and historical NAVs
    for row_index,row in fund_name_2_url_df.iterrows():
        # extract ticker and ID
        fund_ticker=row['fund_ticker']
        fund_id=row['fund_id']
        # define output file names
        holdings_file_name=fund_ticker+'_holdings-'+runDate.strftime('%Y%m%d_%H%M%S')+'.csv'
        #return_file_name=fund_ticker+'-return.csv'
        nav_file_name=fund_ticker+'_nav-'+runDate.strftime('%Y%m%d_%H%M%S')+'.csv'
        # build holdings url
        holdings_csv_url=fetch_fund_holdings_csv_url(fund_ticker,fund_id)
        # build return url
        #return_csv_url=fetch_fund_return_csv_url(fund_ticker,fund_id)
        # build NAV url
        nav_csv_url=fetch_fund_nav_csv_url(fund_ticker,fund_id)
    
        try:
            downloadFile(holdings_csv_url,holdings_directory,holdings_file_name)
        except:
            pass
    
        #try:
        #    downloadFile(return_csv_url,holdings_directory,return_file_name)
        #except:
        #    pass
    
        try:
            downloadFile(nav_csv_url,nav_directory,nav_file_name)
        except:
            pass
        
    return


Configure the path to the Chrome driver:

In [29]:
#driver=webdriver.Chrome('C:/Users/dnokes/chromedriver/chromedriver.exe')
driver=webdriver.Chrome('C:/Users/Derek/Documents/code/python/chromedriver.exe')

Fetch URL for ETF overview .csv and download the .csv:

In [30]:
# build overview soup
overview_soup=build_overview_soup()
# fetch overview CSV URL
overview_csv_url=fetch_overview_csv_url(overview_soup)

In [31]:
overview_csv_url

'https://api.kurtosys.io/tools/ksys317/api/download/csv/overview/all?fileName=BMO_ETF_Overview_Price.csv&locale=en-CA'

In [32]:
# define overview directory and file name
#overview_directory="C:/Users/dnokes/Documents/bmo_demo/overview/"
overview_directory="F:/marketData/global_monitoring/instrument_universe/BMO/overview/"
runDate=datetime.datetime.now()
overview_file_name='etf_overview_price-'+runDate.strftime('%Y%m%d_%H%M%S')+'.csv'
# download
downloadFile(overview_csv_url,overview_directory,overview_file_name)

Build instrument master:

In [33]:
# define base URL
base_url='https://api.kurtosys.io/tools/ksys317/'
# build fund master
fund_name_2_url_df=build_fund_master_df(overview_soup,base_url)
# extract fund ticker
fund_name_2_url_df['fund_ticker']=fund_name_2_url_df['fund_url'].apply(lambda x : extract_fund_ticker(x))
# fetch JSON with fund IDs via web API
d=fetch_json()
# get fund ID for use with web API for each fund ticker
fund_name_2_url_df['fund_id']=fund_name_2_url_df['fund_ticker'].apply(lambda x : fetch_fund_id(x,d))

In [37]:
# remove duplicates
fund_name_2_url_df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

In [38]:
fund_name_2_url_df.style

,fund_name,fund_url,fund_ticker,fund_id
0,BMO Aggregate Bond Index ETF,https://api.kurtosys.io/tools/ksys317/fundProfile/ZAG?locale=en-CA,ZAG,1002156864
1,BMO Discount Bond Index ETF,https://api.kurtosys.io/tools/ksys317/fundProfile/ZDB?locale=en-CA,ZDB,1002156851
2,BMO Short-Term Bond Index ETF,https://api.kurtosys.io/tools/ksys317/fundProfile/ZSB?locale=en-CA,ZSB,1022847837
3,BMO Core Plus Bond Fund,https://api.kurtosys.io/tools/ksys317/fundProfile/ZCPB?locale=en-CA,ZCPB,1024211830
4,BMO Global Multi-Sector Bond Fund,https://api.kurtosys.io/tools/ksys317/fundProfile/ZMSB?locale=en-CA,ZMSB,1024211831
5,BMO Global Strategic Bond Fund,https://api.kurtosys.io/tools/ksys317/fundProfile/ZGSB?locale=en-CA,ZGSB,1024211829
6,BMO Ultra Short-Term Bond ETF,https://api.kurtosys.io/tools/ksys317/fundProfile/ZST?locale=en-CA,ZST,1002156867
7,BMO Ultra Short-Term Bond ETF (Accumulating Units),https://api.kurtosys.io/tools/ksys317/fundProfile/ZST.L?locale=en-CA,ZST.L,1014050431
8,BMO Ultra Short-Term US Bond ETF (USD Units),https://api.kurtosys.io/tools/ksys317/fundProfile/ZUS.U?locale=en-CA,ZUS.U,1028032809
9,BMO Ultra Short-Term US Bond ETF (US Dollar Accumulating Units),https://api.kurtosys.io/tools/ksys317/fundProfile/ZUS.V?locale=en-CA,ZUS.V,1028032803


In [35]:
# define output directories
directory_dict=dict()
directory_dict['holdings']="F:/marketData/global_monitoring/instrument_universe/BMO/holdings/"
directory_dict['nav']="F:/marketData/global_monitoring/instrument_universe/BMO/nav/"
# download fund CSVs
download_fund_CSVs(fund_name_2_url_df,directory_dict)


https://api.kurtosys.io/tools/ksys317/api/download/csv/holding/1002156864?fileName=ZAG_Holdings.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/historicalNav/1002156864?fileName=ZAG_HistoricalNAV.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/holding/1002156851?fileName=ZDB_Holdings.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/historicalNav/1002156851?fileName=ZDB_HistoricalNAV.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/holding/1022847837?fileName=ZSB_Holdings.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/historicalNav/1022847837?fileName=ZSB_HistoricalNAV.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/holding/1024211830?fileName=ZCPB_Holdings.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download/csv/historicalNav/1024211830?fileName=ZCPB_HistoricalNAV.csv&locale=en-CA
https://api.kurtosys.io/tools/ksys317/api/download

Now we fetch some meta data for the funds including the fund descriptions. 

In [36]:
driver.quit()